In [1]:
import pandas as pd
purchase_df = pd.read_csv('data/purchase.csv', sep=',')
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')
condition = (events_df.start_time >= '2018-01-01') & (events_df.start_time < '2019-01-01') & (events_df.event_type =='registration')
registered = events_df[condition]['user_id'].to_list()
events_2018 = events_df[events_df.user_id.isin(registered)]
events_2018.start_time = pd.to_datetime(events_2018.start_time, format='%Y-%m-%dT%H:%M:%S')
purchase_2018 = purchase_df[purchase_df['user_id'].isin(registered)]
purchase_2018.event_datetime = pd.to_datetime(purchase_2018.event_datetime, format='%Y-%m-%dT%H:%M:%S')
registered_users_count = events_2018[events_2018["event_type"] == "registration"][
    "user_id"
].nunique()
tutorial_start_users_count = events_2018[events_2018["event_type"] == "tutorial_start"][
    "user_id"
].nunique()
tutorial_finish_users_count = events_2018[events_2018["event_type"] == "tutorial_finish"][
    "user_id"
].nunique()
level_choice_users_count = events_2018[events_2018["event_type"] == "level_choice"][
    "user_id"
].nunique()
training_choice_users_count = events_2018[events_2018["event_type"] == "pack_choice"][
    "user_id"
].nunique()
paying_users_count = purchase_2018["user_id"].nunique()
percent_of_paying_users = paying_users_count / training_choice_users_count
purchase_2018["event_type"] = 'purchase'
events_2018 = events_2018.rename(columns={"id": "event_id"})
purchase_2018 = purchase_2018.rename(columns={"id": "purchase_id"})
total_events_df = pd.concat([events_2018,purchase_2018],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')
user_path_df = (
    total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index()
)
user_path_df["event_path"] = user_path_df["event_type"].apply(lambda x: " > ".join(x))
user_paths = (
    user_path_df.groupby(["event_path"])["user_id"]
    .nunique()
    .sort_values(ascending=False)
)
registration_df = total_events_df[total_events_df['event_type'] == 'registration']
registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"})
tutorial_start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start']
tutorial_start_df['user_id'].value_counts().mean()
tutorial_start_df_wo_duplicates = tutorial_start_df.sort_values(
    "start_time"
).drop_duplicates("user_id")
tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[
    ["user_id", "tutorial_id", "start_time"]
].rename(columns={"start_time": "tutorial_start_time"})
merged_df = registration_df.merge(tutorial_start_df_wo_duplicates, on="user_id", how="inner")
merged_df["timedelta"] = (merged_df["tutorial_start_time"] - merged_df["registration_time"])
tutorial_finish_df = total_events_df[total_events_df["event_type"] == "tutorial_finish"]
first_tutorial_ids = tutorial_start_df_wo_duplicates["tutorial_id"].unique()
tutorial_finish_df = tutorial_finish_df[
    tutorial_finish_df["tutorial_id"].isin(first_tutorial_ids)]
tutorial_finish_df['user_id'].value_counts().mean()
tutorial_finish_df = tutorial_finish_df[["user_id", "start_time"]].rename(
    columns={"start_time": "tutorial_finish_time"}
)
merged_df_2 = tutorial_start_df_wo_duplicates.merge(
    tutorial_finish_df, on="user_id", how="inner"
)
merged_df_2["timedelta"] = (
    merged_df_2["tutorial_finish_time"] - merged_df_2["tutorial_start_time"]
)
merged_df_2.head()
level_choice_df = total_events_df[total_events_df["event_type"] == "level_choice"]
level_choice_df = level_choice_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"}
)
merged_df_3 = registration_df.merge(level_choice_df, on="user_id", how="inner")
merged_df_3["timedelta"] = (
    merged_df_3["level_choice_time"] - merged_df_3["registration_time"]
)
tutorial_pack_df = total_events_df[total_events_df["event_type"] == "pack_choice"]
tutorial_pack_df = tutorial_pack_df[["user_id", "start_time"]].rename(columns={"start_time": "tutorial_pack_time"})
merged_df_4 = tutorial_pack_df.merge(level_choice_df, on="user_id", how="inner")
merged_df_4["timedelta"] = (
    merged_df_4["tutorial_pack_time"] - merged_df_4["level_choice_time"]
)
first_purchase_df = total_events_df[total_events_df['event_type'] == 'purchase']
first_purchase_df = first_purchase_df[["user_id", "start_time"]].rename(
    columns={"start_time": "first_purchase_time"}
)
merged_df_5 = tutorial_pack_df.merge(first_purchase_df, on="user_id", how="inner")
merged_df_5["timedelta"] = (merged_df_5["first_purchase_time"] - merged_df_5["tutorial_pack_time"]
)
easy = (events_df.selected_level == 'easy')
easy_list = events_df[easy]['user_id'].to_list()
easy_users_count_purchase = purchase_df[purchase_df['user_id'].isin(easy_list)]['user_id'].nunique()
easy_users_count_total = events_df[events_df['user_id'].isin(easy_list)]['user_id'].nunique()
percent_easy_users_count_purchase = easy_users_count_purchase / easy_users_count_total
print("Процент пользователей, которые выбрали уровень сложности easy и совершили оплату (от общего числа пользователей уровня easy): {:.2%}".format(
            percent_easy_users_count_purchase))
medium = (events_df.selected_level == 'medium')
medium_list = events_df[medium]['user_id'].to_list()
medium_users_count_purchase = purchase_df[purchase_df['user_id'].isin(medium_list)]['user_id'].nunique()
medium_users_count_total = events_df[events_df['user_id'].isin(medium_list)]['user_id'].nunique()
percent_medium_users_count_purchase = medium_users_count_purchase / medium_users_count_total
print("Процент пользователей, которые выбрали уровень сложности medium и совершили оплату (от общего числа пользователей уровня medium): {:.2%}".format(
            percent_medium_users_count_purchase))
hard = (events_df.selected_level == 'hard')
hard_list = events_df[hard]['user_id'].to_list()
hard_users_count_purchase = purchase_df[purchase_df['user_id'].isin(hard_list)]['user_id'].nunique()
hard_users_count_total = events_df[events_df['user_id'].isin(hard_list)]['user_id'].nunique()
percent_hard_users_count_purchase = hard_users_count_purchase / hard_users_count_total
print("Процент пользователей, которые выбрали уровень сложности hard и совершили оплату (от общего числа пользователей уровня hard): {:.2%}".format(
            percent_hard_users_count_purchase))
merged_df_6 = registration_df.merge(first_purchase_df, on="user_id", how="inner")
merged_df_6["timedelta"] = (merged_df_6["first_purchase_time"] - merged_df_6["registration_time"])
merged_df_6.head()
print(merged_df_6["timedelta"].mean())
print(merged_df_6["timedelta"].describe())

/var/folders/2t/41ymw9416szdry3f82rfxvr80000gn/T/ipykernel_88431/4040552569.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_2018.start_time = pd.to_datetime(events_2018.start_time, format='%Y-%m-%dT%H:%M:%S')
/var/folders/2t/41ymw9416szdry3f82rfxvr80000gn/T/ipykernel_88431/4040552569.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_2018.event_datetime = pd.to_datetime(purchase_2018.event_datetime, format='%Y-%m-%dT%H:%M:%S')
/var/folders/2t/41ymw9416szdry3f82rfxvr80000gn/T/ipykernel_8

Процент пользователей, которые выбрали уровень сложности easy и совершили оплату (от общего числа пользователей уровня easy): 8.52%
Процент пользователей, которые выбрали уровень сложности medium и совершили оплату (от общего числа пользователей уровня medium): 20.35%
Процент пользователей, которые выбрали уровень сложности hard и совершили оплату (от общего числа пользователей уровня hard): 35.25%
NaT
count      0
mean     NaT
std      NaT
min      NaT
25%      NaT
50%      NaT
75%      NaT
max      NaT
Name: timedelta, dtype: object
